
# アプリケーションの評価と改善

https://docs.databricks.com/aws/ja/mlflow3/genai/eval-monitor/evaluate-app

In [0]:
%pip install "mlflow[databricks]>=3.1.1" openai

%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## ステップ 1: アプリケーションを作成する

このガイドでは、次のような Eメール 生成アプリを評価します。

- CRMデータベースから顧客情報を取得します
- 取得した情報に基づいてパーソナライズされたフォローアップEメール

Eメール生成アプリを作りましょう。 取得コンポーネントは、MLflow の取得固有のスコアラーを有効にするために ```span_type="RETRIEVER"``` でマークされています。

In [0]:
import mlflow
from openai import OpenAI
from mlflow.entities import Document
from typing import List, Dict

# OpenAI呼び出しの自動トレースを有効にする
mlflow.openai.autolog()

# 同じ資格情報を使用してOpenAI経由でDatabricks LLMに接続する
# あるいは、ここで独自のOpenAI資格情報を使用することもできます
mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
client = OpenAI(
    api_key=mlflow_creds.token, base_url=f"{mlflow_creds.host}/serving-endpoints"
)

# シミュレートされたCRMデータベース
CRM_DATA = {
    "Acme Corp": {
        "contact_name": "アリス・チェン",
        "recent_meeting": "月曜日に製品デモを行い、エンタープライズ機能に非常に興味を持っていました。彼らは次のことについて質問しました：高度な分析、リアルタイムダッシュボード、API統合、カスタムレポート、マルチユーザーサポート、SSO認証、データエクスポート機能、および500人以上のユーザー向けの価格設定",
        "support_tickets": [
            "Ticket #123: APIの遅延問題（先週解決済み）",
            "Ticket #124: 一括インポートの機能リクエスト",
            "Ticket #125: GDPRコンプライアンスに関する質問",
        ],
        "account_manager": "サラ・ジョンソン",
    },
    "TechStart": {
        "contact_name": "ボブ・マルティネス",
        "recent_meeting": "先週の木曜日に初回の営業電話を行い、価格をリクエストしました",
        "support_tickets": [
            "Ticket #456: ログイン問題（オープン - クリティカル）",
            "Ticket #457: パフォーマンスの低下が報告されました",
            "Ticket #458: 彼らのCRMとの統合が失敗しています",
        ],
        "account_manager": "マイク・トンプソン",
    },
    "Global Retail": {
        "contact_name": "キャロル・ワン",
        "recent_meeting": "昨日の四半期レビューで、プラットフォームのパフォーマンスに満足しています",
        "support_tickets": [],
        "account_manager": "サラ・ジョンソン",
    },
}


# MLflowの事前定義されたRetrievalGroundednessスコアラーが機能するようにリトリーバースパンを使用する
@mlflow.trace(span_type="RETRIEVER")
def retrieve_customer_info(customer_name: str) -> List[Document]:
    """CRMデータベースから顧客情報を取得する"""
    if customer_name in CRM_DATA:
        data = CRM_DATA[customer_name]
        return [
            Document(
                id=f"{customer_name}_meeting",
                page_content=f"Recent meeting: {data['recent_meeting']}",
                metadata={"type": "meeting_notes"},
            ),
            Document(
                id=f"{customer_name}_tickets",
                page_content=f"Support tickets: {', '.join(data['support_tickets']) if data['support_tickets'] else 'No open tickets'}",
                metadata={"type": "support_status"},
            ),
            Document(
                id=f"{customer_name}_contact",
                page_content=f"Contact: {data['contact_name']}, Account Manager: {data['account_manager']}",
                metadata={"type": "contact_info"},
            ),
        ]
    return []


@mlflow.trace
def generate_sales_email(customer_name: str, user_instructions: str) -> Dict[str, str]:
    """顧客データと営業担当者の指示に基づいてパーソナライズされた営業メールを生成する"""
    # 顧客情報を取得する
    customer_docs = retrieve_customer_info(customer_name)

    # 取得したコンテキストを結合する
    context = "\n".join([doc.page_content for doc in customer_docs])

    # 取得したコンテキストを使用してメールを生成する
    prompt = f"""あなたは営業担当者です。以下の顧客情報に基づき、
    顧客からのリクエストに対応した簡潔なフォローアップメールを作成してください。

    顧客情報:
    {context}

    ユーザー指示: {user_instructions}

    メールは簡潔かつパーソナライズされた内容にしてください。"""

    response = client.chat.completions.create(
        model="databricks-meta-llama-3-3-70b-instruct",
        messages=[
            {"role": "system", "content": "あなたは役に立つ営業アシスタントです。"},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
    )

    return {"email": response.choices[0].message.content}


# アプリケーションをテストする
result = generate_sales_email("Acme Corp", "商品デモの後にフォローアップして")
print(result["email"])

以下は、商品デモの後にアリスさんへフォローアップするメールの例です。

---

件名: エンタープライズ機能に関するフォローアップ

アリスさん

先日は、当社の製品デモにご参加いただき、誠にありがとうございました。エンタープライズ機能に対するご関心を記憶しています。この度、ご質問いただいた機能やサポートチケットに関する現在の状況についてご報告いたします。

1. **高度な分析とリアルタイムダッシュボード**：当社のソリューションは、詳細な分析とリアルタイムのデータ提供を可能とします。これにより、ビジネスのインサイトを深め、迅速な意思決定をサポートします。
2. **API統合**：APIを通じての統合は、システム間のシームレスなデータ交換を可能にします。この機能については、先週解決したAPIの遅延問題（Ticket #123）以来、さらなる最適化が行われています。
3. **カスタムレポートとマルチユーザーサポート**：ご要望に応え、カスタマイズ可能なレポート機能と複数ユーザーのサポートが提供可能です。これにより、チームのコラボレーションとレポートのカスタマイズが容易になります。
4. **SSO認証とデータエクスポート**：シングルサインオン（SSO）認証とデータエクスポート機能も提供しています。これらは、セキュリティとデータの可搬性を向上させるための重要な機能です。
5. **500人以上のユーザー向けの価格設定**：ご要望に応じて、エンタープライズプランの価格について詳細を提供します。請求に関する詳細な情報を別途送付します。

また、ご要望のあった一括インポートに関する機能リクエスト（Ticket #124）とGDPRコンプライアンスに関する質問（Ticket #125）についても、担当チームが対応中です。ご了承ください。

アカウントマネージャーのサラ・ジョンソンもご連絡先をご共有しております。ご不明な点や追加の質問がございましたら、いつでもお気軽にご連絡ください。サラはあなたのニーズに合わせたサポートを提供いたします。

今後ともどうぞよろしくお願い申し上げます。

---
[あなたの名前]  
営業担当  
[会社名]  
[連絡先]


Trace(trace_id=tr-80cb0e09a84eaa29f91d7d659d0de83d)


## ステップ 2: 本番運用のトラフィックをシミュレートする

この手順では、デモンストレーションの目的でトラフィックをシミュレートします。

実際には、実際の使用状況のトレース トレース を使用して評価データセットを作成します。

In [0]:
# トレース管理をするLoggedModelを設定
active_model_info = mlflow.set_active_model(name="sales_email_v1")


2025/06/29 01:45:17 INFO mlflow.tracking.fluent: LoggedModel with name 'sales_email_v1' does not exist, creating one...
2025/06/29 01:45:19 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-8333440af4ba404ebc5c96a51a814ac4


In [0]:
# ガイドライン違反を意図したシナリオでベータテストのトラフィックをシミュレート
test_requests = [
    {"customer_name": "Acme Corp", "user_instructions": "製品デモの後にフォローアップ"},
    {"customer_name": "TechStart", "user_instructions": "サポートチケットの状況を確認"},
    {"customer_name": "Global Retail", "user_instructions": "四半期レビューの要約を送信"},
    {"customer_name": "Acme Corp", "user_instructions": "すべての製品機能、価格帯、実装スケジュール、およびサポートオプションを詳しく説明する非常に詳細なメールを書く"},
    {"customer_name": "TechStart", "user_instructions": "彼らのビジネスに感謝する熱意のあるメールを送信"},
    {"customer_name": "Global Retail", "user_instructions": "フォローアップメールを送信"},
    {"customer_name": "Acme Corp", "user_instructions": "物事がどうなっているかを確認するために連絡"},
]

# リクエストを実行してトレースを記録する
print("プロダクション トラフィックをシミュレートしています...")
for req in test_requests:
    try:
        result = generate_sales_email(**req)
        print(f"✓ {req['customer_name']} のメールを生成しました")
    except Exception as e:
        print(f"✗ {req['customer_name']} のエラー: {e}")

プロダクション トラフィックをシミュレートしています...
✓ Acme Corp のメールを生成しました
✓ TechStart のメールを生成しました
✓ Global Retail のメールを生成しました
✓ Acme Corp のメールを生成しました
✓ TechStart のメールを生成しました
✓ Global Retail のメールを生成しました
✓ Acme Corp のメールを生成しました


[Trace(trace_id=tr-98333ad260ef2fb7777f30921ca76742), Trace(trace_id=tr-10d380ae4a8c4d36ff39de4218c8a42b), Trace(trace_id=tr-14a88afefe5a2603d7beaf541064ffb2), Trace(trace_id=tr-5c352ec8116a3626d8ee0dd4dfc7b4bf), Trace(trace_id=tr-d043186e5620d03e8be5df1df7451705), Trace(trace_id=tr-563b9e6a26b383e82267a3a2dea48392), Trace(trace_id=tr-7df5143f899aecba556793347493fd69)]

## ステップ 3: 評価データセットを作成する

次に、トレースを評価データセットに変換しましょう。評価データセットにトレースを保存すると、評価結果をデータセットにリンクして、データセットの経時的な変更を追跡し、このデータセットを使用して生成されたすべての評価結果を確認できます。

評価データセットをプログラムで作成するには、トレースを検索し、それらをデータセットに追加します。

In [0]:
mlflow.get_active_model_id()

'm-8333440af4ba404ebc5c96a51a814ac4'

In [0]:
import mlflow
import mlflow.genai.datasets
import time

# 1. 評価データセットを作成

uc_schema = "workspace.default"
evaluation_dataset_table_name = "email_generation_eval"

# すでにデータセットがある場合は削除
try:
    mlflow.genai.datasets.delete_dataset(
        uc_table_name=f"{uc_schema}.{evaluation_dataset_table_name}",
    )
except Exception as e:
    print(f"評価データセットは存在しません: {e}")

# データセットを作成
eval_dataset = mlflow.genai.datasets.create_dataset(
    uc_table_name=f"{uc_schema}.{evaluation_dataset_table_name}",
)
print(f"評価データセットを作成しました: {uc_schema}.{evaluation_dataset_table_name}")

# 2. sales_email_v1のトレースを全権取得
traces = mlflow.search_traces(
    model_id=mlflow.get_active_model_id(),
    order_by=["attributes.timestamp_ms DESC"],
)

print(f"エクスペリメントから {len(traces)} 件の成功したトレースが見つかりました")

# 3. トレースを評価データセットに追加
eval_dataset.merge_records(traces)
print(f"{len(traces)} 件のレコードを評価データセットに追加しました")
# データセットのプレビュー
df = eval_dataset.to_df()
print(f"\nデータセットのプレビュー:")
print(f"総レコード数: {len(df)}")
print("\nサンプルレコード:")
sample = df.iloc[0]
print(f"Inputs: {sample['inputs']}")

評価データセットを作成しました: workspace.default.email_generation_eval
エクスペリメントから 7 件の成功したトレースが見つかりました
7 件のレコードを評価データセットに追加しました

データセットのプレビュー:
総レコード数: 7

サンプルレコード:
Inputs: {'customer_name': 'Acme Corp', 'user_instructions': 'すべての製品機能、価格帯、実装スケジュール、およびサポートオプションを詳しく説明する非常に詳細なメールを書く'}


In [0]:
eval_dataset = mlflow.genai.datasets.get_dataset(
    uc_table_name=f"{uc_schema}.{evaluation_dataset_table_name}",
)

display(eval_dataset.to_df())

dataset_record_id,inputs,expectations,source,tags,create_time,last_update_time,created_by,last_updated_by
4a2f5f1f-f157-43e0-a277-96230ee56b48,"List(Acme Corp, すべての製品機能、価格帯、実装スケジュール、およびサポートオプションを詳しく説明する非常に詳細なメールを書く)",List(),"List(null, null, List(tr-5c352ec8116a3626d8ee0dd4dfc7b4bf))","List(dbfs:/databricks/mlflow-tracking/3f14e493c7af4ee4b7756305063051c9/tr-5c352ec8116a3626d8ee0dd4dfc7b4bf/artifacts, generate_sales_email, isanakamishiro@gmail.com)",2025-06-29T02:16:18.299Z,2025-06-29T02:16:18.299Z,isanakamishiro@gmail.com,isanakamishiro@gmail.com
59b2b9b0-de7e-4d35-b7e6-3e9246e11914,"List(Global Retail, フォローアップメールを送信)",List(),"List(null, null, List(tr-563b9e6a26b383e82267a3a2dea48392))","List(dbfs:/databricks/mlflow-tracking/3f14e493c7af4ee4b7756305063051c9/tr-563b9e6a26b383e82267a3a2dea48392/artifacts, generate_sales_email, isanakamishiro@gmail.com)",2025-06-29T02:16:18.299Z,2025-06-29T02:16:18.299Z,isanakamishiro@gmail.com,isanakamishiro@gmail.com
5c49a362-a9ab-44f7-a588-8230a8ec99b1,"List(Acme Corp, 物事がどうなっているかを確認するために連絡)",List(),"List(null, null, List(tr-7df5143f899aecba556793347493fd69))","List(dbfs:/databricks/mlflow-tracking/3f14e493c7af4ee4b7756305063051c9/tr-7df5143f899aecba556793347493fd69/artifacts, generate_sales_email, isanakamishiro@gmail.com)",2025-06-29T02:16:18.299Z,2025-06-29T02:16:18.299Z,isanakamishiro@gmail.com,isanakamishiro@gmail.com
7ddb20a5-3c1c-4cf2-9dbf-bb3f7fdc6051,"List(TechStart, 彼らのビジネスに感謝する熱意のあるメールを送信)",List(),"List(null, null, List(tr-d043186e5620d03e8be5df1df7451705))","List(dbfs:/databricks/mlflow-tracking/3f14e493c7af4ee4b7756305063051c9/tr-d043186e5620d03e8be5df1df7451705/artifacts, generate_sales_email, isanakamishiro@gmail.com)",2025-06-29T02:16:18.299Z,2025-06-29T02:16:18.299Z,isanakamishiro@gmail.com,isanakamishiro@gmail.com
8f053e6f-f7e7-47f1-a17f-09b87415d7b4,"List(TechStart, サポートチケットの状況を確認)",List(),"List(null, null, List(tr-10d380ae4a8c4d36ff39de4218c8a42b))","List(dbfs:/databricks/mlflow-tracking/3f14e493c7af4ee4b7756305063051c9/tr-10d380ae4a8c4d36ff39de4218c8a42b/artifacts, generate_sales_email, isanakamishiro@gmail.com)",2025-06-29T02:16:18.299Z,2025-06-29T02:16:18.299Z,isanakamishiro@gmail.com,isanakamishiro@gmail.com
98d07009-3609-4dfe-b3fb-dd1370a159cb,"List(Acme Corp, 製品デモの後にフォローアップ)",List(),"List(null, null, List(tr-98333ad260ef2fb7777f30921ca76742))","List(dbfs:/databricks/mlflow-tracking/3f14e493c7af4ee4b7756305063051c9/tr-98333ad260ef2fb7777f30921ca76742/artifacts, generate_sales_email, isanakamishiro@gmail.com)",2025-06-29T02:16:18.299Z,2025-06-29T02:16:18.299Z,isanakamishiro@gmail.com,isanakamishiro@gmail.com
fe3a08dd-b851-4a8a-abcc-61440f3b8a8d,"List(Global Retail, 四半期レビューの要約を送信)",List(),"List(null, null, List(tr-14a88afefe5a2603d7beaf541064ffb2))","List(dbfs:/databricks/mlflow-tracking/3f14e493c7af4ee4b7756305063051c9/tr-14a88afefe5a2603d7beaf541064ffb2/artifacts, generate_sales_email, isanakamishiro@gmail.com)",2025-06-29T02:16:18.299Z,2025-06-29T02:16:18.299Z,isanakamishiro@gmail.com,isanakamishiro@gmail.com


Databricks visualization. Run in Databricks to view.


## ステップ4:事前定義されたスコアラーで評価を実行する[​](#ステップ4事前定義されたスコアラーで評価を実行�する "ステップ4事前定義されたスコアラーで評価を実行する への直接リンク")

次に、MLflow に用意されている [定義済みのスコアラー](/aws/ja/mlflow3/genai/eval-monitor/concepts/judges/pre-built-judges-scorers) を使用して、生成AI アプリケーションの品質のさまざまな側面を自動的に評価してみましょう。詳細については、 [LLM ベースのスコアラー](/aws/ja/mlflow3/genai/eval-monitor/concepts/judges/) と [コードベースのスコアラー](/aws/ja/mlflow3/genai/eval-monitor/concepts/scorers) のリファレンスページを参照してください。

注記

必要に応じて、MLflow を使用してアプリケーションとプロンプトのバージョンを追跡できます。詳細については、 [トラック アプリとプロンプト バージョンの](/aws/ja/mlflow3/genai/prompt-version-mgmt/prompt-registry/track-prompts-app-versions) ガイドをご覧ください。

In [0]:
from mlflow.genai.scorers import (
    RetrievalGroundedness,
    RelevanceToQuery,
    Safety,
    Guidelines,
)

# スコアラーを変数として保存し、ステップ7で再利用できるようにします

email_scorers = [
    RetrievalGroundedness(),  # メール内容が取得したデータに基づいているかをチェック
    Guidelines(
        name="follows_instructions",
        guidelines="生成されたメールは、リクエスト内のuser_instructions（ユーザー指示）に従っている必要があります。",
    ),
    Guidelines(
        name="concise_communication",
        guidelines="メールは必ず簡潔かつ要点を押さえている必要があります。重要な文脈を失うことなく、主要なメッセージを効率的に伝えるべきです。",
    ),
    Guidelines(
        name="mentions_contact_name",
        guidelines="メールの挨拶文で、顧客担当者のファーストネーム（例：Alice、Bob、Carol）を明示的に記載する必要があります。「Hello」や「Dear Customer」などの一般的な挨拶は不可とします。",
    ),
    Guidelines(
        name="professional_tone",
        guidelines="メールはプロフェッショナルな口調で書かれている必要があります。",
    ),
    Guidelines(
        name="includes_next_steps",
        guidelines="メールの最後に、具体的かつ実行可能な次のアクションと明確なタイムラインを必ず記載してください。",
    ),
    RelevanceToQuery(),  # メールがユーザーのリクエストに対応しているかをチェック
    Safety(),  # 有害または不適切な内容が含まれていないかをチェック
]

# 定義済みスコアラーで評価を実行
eval_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=generate_sales_email,
    scorers=email_scorers,
)

2025/06/29 02:23:51 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset.
2025/06/29 02:24:02 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-8333440af4ba404ebc5c96a51a814ac4
2025/06/29 02:24:02 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/29 02:24:03 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.


Evaluating:   0%|          | 0/7 [Elapsed: 00:00, Remaining: ?] 

<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 
 View evaluation results.

[Trace(trace_id=tr-f383143bca179fe41e90769f361cc33d), Trace(trace_id=tr-f975a936f29b5a4771fe1e4741165384), Trace(trace_id=tr-b8fee8e33925b8ce47c0902fe82a097a), Trace(trace_id=tr-c2900e95bbc4a2a1f68fb45d7bc71321), Trace(trace_id=tr-68d82a14d9ec188f364155ba9cea1469), Trace(trace_id=tr-8ccc7fcefcc1ec32ab476f71bf5944f3), Trace(trace_id=tr-f61047a886c18bdc4a4faa8b7dcb3502)]

In [0]:
eval_results.tables["eval_results"]

,trace_id,trace,client_request_id,state,request_time,execution_duration,request,response,trace_metadata,tags,spans,assessments
0,tr-f61047a886c18bdc4a4faa8b7dcb3502,{'data': {'intermediate_outputs': {'retrieve_c...,tr-f61047a886c18bdc4a4faa8b7dcb3502,OK,2025-06-29 02:24:03.406,4126,"{'customer_name': 'Global Retail', 'user_instr...",{'email': '以下はフォローアップメールの例です。 --- Subject: 四...,{'mlflow.sourceRun': 'bd87f5088b104d658b9c5537...,{'mlflow.artifactLocation': 'dbfs:/databricks/...,"[{'trace_id': '9hBHqIbBi9xKT6qLfcs1Ag==', 'spa...",[{'assessment_id': 'a-1590d61ea57e4edfa0861255...
1,tr-f975a936f29b5a4771fe1e4741165384,{'data': {'intermediate_outputs': {'retrieve_c...,tr-f975a936f29b5a4771fe1e4741165384,OK,2025-06-29 02:24:03.404,4682,"{'customer_name': 'Acme Corp', 'user_instructi...",{'email': '以下のようなメールを作成します。 メール本文： アリスさん 月曜...,{'mlflow.sourceRun': 'bd87f5088b104d658b9c5537...,{'mlflow.artifactLocation': 'dbfs:/databricks/...,"[{'trace_id': '+XWpNvKbWkdx/h5HQRZThA==', 'spa...",[{'assessment_id': 'a-e9103af0bd154d75abff425e...
2,tr-c2900e95bbc4a2a1f68fb45d7bc71321,{'data': {'intermediate_outputs': {'retrieve_c...,tr-c2900e95bbc4a2a1f68fb45d7bc71321,OK,2025-06-29 02:24:03.403,3994,"{'customer_name': 'TechStart', 'user_instructi...",{'email': 'Here's a concise and personalized f...,{'mlflow.sourceRun': 'bd87f5088b104d658b9c5537...,{'mlflow.artifactLocation': 'dbfs:/databricks/...,"[{'trace_id': 'wpAOlbvEoqH2j7Rde8cTIQ==', 'spa...",[{'assessment_id': 'a-9374647417d94ab885d130e2...
3,tr-b8fee8e33925b8ce47c0902fe82a097a,{'data': {'intermediate_outputs': {'retrieve_c...,tr-b8fee8e33925b8ce47c0902fe82a097a,OK,2025-06-29 02:24:03.402,5064,"{'customer_name': 'Acme Corp', 'user_instructi...",{'email': '以下はフォローアップメールの例です。 サラ・ジョンソン アカウントマ...,{'mlflow.sourceRun': 'bd87f5088b104d658b9c5537...,{'mlflow.artifactLocation': 'dbfs:/databricks/...,"[{'trace_id': 'uP7o4zkluM5HwJAv6CoJeg==', 'spa...",[{'assessment_id': 'a-5764b49b9f6e46e78ee13dc0...
4,tr-8ccc7fcefcc1ec32ab476f71bf5944f3,{'data': {'intermediate_outputs': {'retrieve_c...,tr-8ccc7fcefcc1ec32ab476f71bf5944f3,OK,2025-06-29 02:24:03.402,3897,"{'customer_name': 'TechStart', 'user_instructi...",{'email': 'Subject: 感謝の気持ちとフォローアップ ボブさん いつも弊...,{'mlflow.sourceRun': 'bd87f5088b104d658b9c5537...,{'mlflow.artifactLocation': 'dbfs:/databricks/...,"[{'trace_id': 'jMx/zvzB7DKrR29xv1lE8w==', 'spa...",[{'assessment_id': 'a-c6ada064c5e64ad295fed237...
5,tr-f383143bca179fe41e90769f361cc33d,{'data': {'intermediate_outputs': {'retrieve_c...,tr-f383143bca179fe41e90769f361cc33d,OK,2025-06-29 02:24:03.401,2634,"{'customer_name': 'Global Retail', 'user_instr...",{'email': '以下はフォローアップメールの例です。 主题：四半期レビューのフォロー...,{'mlflow.sourceRun': 'bd87f5088b104d658b9c5537...,{'mlflow.artifactLocation': 'dbfs:/databricks/...,"[{'trace_id': '84MUO8oXn+QekHafNhzDPQ==', 'spa...",[{'assessment_id': 'a-4a97c1223e79450e956743dd...
6,tr-68d82a14d9ec188f364155ba9cea1469,{'data': {'intermediate_outputs': {'retrieve_c...,tr-68d82a14d9ec188f364155ba9cea1469,OK,2025-06-29 02:24:03.400,10626,"{'customer_name': 'Acme Corp', 'user_instructi...",{'email': 'アリスさん 先日は月曜日の製品デモに参加いただき、ありがとうございま...,{'mlflow.sourceRun': 'bd87f5088b104d658b9c5537...,{'mlflow.artifactLocation': 'dbfs:/databricks/...,"[{'trace_id': 'aNgqFNnsGI82QVW6nOoUaQ==', 'spa...",[{'assessment_id': 'a-60676f6420044846896ce421...


## ステップ 5: 結果の表示と解釈[​](#ステップ-5-結果の表示と解釈 "ステップ-5-結果の表示と解釈 への直接リンク")

`mlflow.genai.evaluate()`を実行すると、評価データセット内のすべての行の[トレース](/aws/ja/mlflow3/genai/tracing/)と、各スコアラーからの[フィードバック](/aws/ja/mlflow3/genai/tracing/data-model#feedback)が関連付けられた評価ランが作成されます。

評価ランを使用して、次のことを行います。

* **集計メトリクスの参照** : それぞれのスコアラーのすべてのテストケースにおける平均パフォーマンス
* **個々の障害ケースのデバッグ** : 障害が発生した理由を理解し、将来のバージョンで行うべき改善点を特定します
* **故障解析** :採点者が課題を特定した具体例

この評価では、いくつかの問題が見られます。

1. **不適切な指示フォロー** - エージェントは、簡単なチェックインを求められたときに詳細な製品情報を送信したり、熱心なお礼のメッセージを求められたときにサポートチケットの更新を提供したりするなど、ユーザーのリクエストと一致しない応答を頻繁に提供します
2. **簡潔さの欠如** - ほとんどのEメールは不必要に長く、重要なメッセージを薄めるほど詳細が多すぎて、Eメールを「簡潔でパーソナライズ」に保つように指示されているにもかかわらず、効率的にコミュニケーションをとることができません。
3. **具体的な次のステップが欠けている** - Eメールの大部分は、必須要素として特定された具体的なタイムラインを含む、具体的で実行可能な次のステップで終わらない

* Using the UI* Using the SDK

MLflow UI の [評価] タブから評価結果にアクセスし、アプリケーションのパフォーマンスを理解します。

![trace](https://assets.docs.databricks.com/_static/images/mlflow3-genai/new-images/eval-guide-results.gif)

In [0]:
# 評価実行のトレースを取得
eval_traces = mlflow.search_traces(run_id=eval_results.run_id)

# eval_tracesは評価されたトレースを含むPandas DataFrameです。`assessments`列には各スコアラーのフィードバックが含まれます。
print(eval_traces)

                              trace_id  ...                                        assessments
0  tr-f61047a886c18bdc4a4faa8b7dcb3502  ...  [Feedback(name='safety', source=AssessmentSour...
1  tr-f975a936f29b5a4771fe1e4741165384  ...  [Feedback(name='includes_next_steps', source=A...
2  tr-c2900e95bbc4a2a1f68fb45d7bc71321  ...  [Feedback(name='professional_tone', source=Ass...
3  tr-b8fee8e33925b8ce47c0902fe82a097a  ...  [Feedback(name='concise_communication', source...
4  tr-8ccc7fcefcc1ec32ab476f71bf5944f3  ...  [Feedback(name='agent/total_token_count', sour...
5  tr-f383143bca179fe41e90769f361cc33d  ...  [Feedback(name='retrieval_groundedness', sourc...
6  tr-68d82a14d9ec188f364155ba9cea1469  ...  [Feedback(name='relevance_to_query', source=As...

[7 rows x 12 columns]


## ステップ 6: 改良版を作成する[​](#ステップ-6-改良版を作成する "ステップ-6-改良版を作成する への直接リンク")

評価結果に基づいて、特定された問題に対処する改善バージョンを作成しましょう。

注記

新しいバージョンの `generate_sales_email()` 関数では、最初のステップから `retrieve_customer_info()` 取得した関数を使用します。


In [0]:
# 新しいLoggedModelを作成・設定
active_model_info = mlflow.set_active_model(name="sales_email_v2")

@mlflow.trace
def generate_sales_email_v2(
    customer_name: str, user_instructions: str
) -> Dict[str, str]:
    """顧客データと営業担当者の指示に基づいてパーソナライズされた営業メールを生成します。"""
    # 顧客情報を取得
    customer_docs = retrieve_customer_info(customer_name)

    if not customer_docs:
        return {"error": f"{customer_name}の顧客データが見つかりません"}

    # 取得したコンテキストを結合
    context = "\n".join([doc.page_content for doc in customer_docs])

    # より良い指示に従ってメールを生成
    prompt = f"""あなたは営業担当者です。メールを書いてください。

最も重要なこと: 以下のユーザー指示に正確に従ってください:
{user_instructions}

顧客コンテキスト（指示に関連するもののみ使用してください）:
{context}

ガイドライン:
1. ユーザー指示を最優先にしてください
2. メールは簡潔に - ユーザーのリクエストに直接関連する情報のみを含めてください
3. 具体的で実行可能な次のステップを明確なタイムラインと共にメールの最後に記載してください（例：「金曜日までに価格をフォローアップします」や「今週15分の電話をスケジュールしましょう」）
4. ユーザーの指示に直接関連する場合のみ顧客情報を参照してください

ユーザーの正確なリクエストを満たす簡潔で焦点を絞ったメールを書いてください。"""

    response = client.chat.completions.create(
        model="databricks-meta-llama-3-3-70b-instruct",
        messages=[
            {
                "role": "system",
                "content": "あなたは簡潔で指示に焦点を当てたメールを書く役立つ営業アシスタントです。",
            },
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
    )

    return {"email": response.choices[0].message.content}

# アプリケーションをテスト
result = generate_sales_email("Acme Corp", "製品デモの後にフォローアップしてください")
print(result["email"])

2025/06/29 02:47:47 INFO mlflow.tracking.fluent: LoggedModel with name 'sales_email_v2' does not exist, creating one...
2025/06/29 02:47:48 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-0e841ae6ba4f4e9ab4b749fdb4827a83


件名：月曜日の製品デモのフォローアップ

Dear アリスさん

月曜日に私たちと一緒に製品デモに参加いただき、ありがとうございました。我々は、エンタープライズ機能、特に高度な分析、リアルタイムダッシュボード、API統合、カスタムレポート、マルチユーザーサポート、SSO認証、データエクスポート機能、および500人以上のユーザー向けの価格設定についてのご関心を大変嬉しく思いました。

Ticket #123 については、先週 API の遅延問題を解決したことをご存じでしょう。Ticket #124 および Ticket #125 については、カスタマーサポートチームがご要望の_one bulk import 機能リクエストおよび GDPR コンプライアンスに関する質問_に関してご連絡を差し上げる予定です。

アカウントマネージャーのサラは、私たちにご要望をより深く理解する機会を与えてくださり、感謝しております。この製品デモの後、追加の質問や懸念事項がございましたら、お気軽にお問い合わせください。

私たちがどのようにお役に立てるかを知りたい場合は、こちらのリンクをクリックしてください: [リンク]

私はあなたのビジネスを成功させるために私たちがどのように役に立つかを理解するためにここにいます。何か質問や懸念がある場合は、いつでも私に連絡してください。

ご健康と幸せを祈ります。

ベストリーガーズ
[あなたの名前]
顧客サポート担当


Trace(trace_id=tr-91bca1d6cd34fbf3efd6c1b9b7610df5)

In [0]:
import mlflow
# 新しいバージョンの評価を、以前と同じスコアラーを使用して実行します
# start_runを使用してUIで評価ランを名前付けします
with mlflow.start_run(run_name="v2"):
    eval_results_v2 = mlflow.genai.evaluate(
        data=eval_dataset, # 同じデータセット
        predict_fn=generate_sales_email_v2, # 新しいアプリバージョンの関数
        scorers=email_scorers, # 前回と同じスコアラーを利用
    )

2025/06/29 02:52:05 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset.
2025/06/29 02:52:16 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-0e841ae6ba4f4e9ab4b749fdb4827a83
2025/06/29 02:52:16 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.


Evaluating:   0%|          | 0/7 [Elapsed: 00:00, Remaining: ?] 

<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 
 View evaluation results.

[Trace(trace_id=tr-6bfcd1e733800ae0c701e76ca635b19b), Trace(trace_id=tr-27d7287c092876b08e04119ee7bbfee1), Trace(trace_id=tr-73647734d0880db558689485f47c992f), Trace(trace_id=tr-6c26db10348a285d0b6f1863ae7a5915), Trace(trace_id=tr-f49d1c2765dfe1dcfcc3109d126c41ff), Trace(trace_id=tr-5bfea6ce91a538dac1e52ef3e593fab4), Trace(trace_id=tr-bc4214c182168cb1f3f62e70f16fd26c)]

## 比較する

In [0]:
import pandas as pd

# mlflow.search_runsはINやOR演算子をサポートしていないため、個別にランを取得
run_v1_df = mlflow.search_runs(filter_string=f"run_id = '{eval_results.run_id}'")
run_v2_df = mlflow.search_runs(filter_string=f"run_id = '{eval_results_v2.run_id}'")

# メトリクス列を抽出（.aggregate_scoreではなく/meanで終わるもの）
# 品質比較のため、エージェント系メトリクス（latency, token counts）は除外
metric_cols = [
    col
    for col in run_v1_df.columns
    if col.startswith("metrics.") and col.endswith("/mean") and "agent/" not in col
]

# 比較テーブルを作成
comparison_data = []
for metric in metric_cols:
    metric_name = metric.replace("metrics.", "").replace("/mean", "")
    v1_score = run_v1_df[metric].iloc[0]
    v2_score = run_v2_df[metric].iloc[0]
    improvement = v2_score - v1_score

    comparison_data.append(
        {
            "Metric": metric_name,
            "V1 Score": f"{v1_score:.3f}",
            "V2 Score": f"{v2_score:.3f}",
            "Improvement": f"{improvement:+.3f}",
            "Improved": "✓" if improvement >= 0 else "✗",
        }
    )

comparison_df = pd.DataFrame(comparison_data)
print("\n=== バージョン比較結果 ===")
print(comparison_df.to_string(index=False))

# 全体の平均改善度を計算（品質メトリクスのみ対象）
avg_v1 = run_v1_df[metric_cols].mean(axis=1).iloc[0]
avg_v2 = run_v2_df[metric_cols].mean(axis=1).iloc[0]
print(
    f"\n全体平均の改善度: {(avg_v2 - avg_v1):+.3f} ({((avg_v2/avg_v1 - 1) * 100):+.1f}%)"
)


=== Version Comparison Results ===
                Metric V1 Score V2 Score Improvement Improved
     professional_tone    1.000    1.000      +0.000        ✓
  follows_instructions    0.667    0.600      -0.067        ✗
   includes_next_steps    0.000    0.800      +0.800        ✓
 mentions_contact_name    1.000    1.000      +0.000        ✓
retrieval_groundedness    1.000    0.600      -0.400        ✗
 concise_communication    0.333    0.600      +0.267        ✓
    relevance_to_query    0.833    1.000      +0.167        ✓

Overall average improvement: +0.110 (+15.9%)


In [0]:
import pandas as pd
# 両バージョンの詳細なトレースを取得
traces_v1 = mlflow.search_traces(run_id=eval_results.run_id)
traces_v2 = mlflow.search_traces(run_id=eval_results_v2.run_id)

# 入力パラメータに基づいてマージキーを作成
traces_v1['merge_key'] = traces_v1['request'].apply(
    lambda x: f"{x.get('customer_name', '')}|{x.get('user_instructions', '')}"
)
traces_v2['merge_key'] = traces_v2['request'].apply(
    lambda x: f"{x.get('customer_name', '')}|{x.get('user_instructions', '')}"
)

# 同じ入力データでマージして比較
merged = traces_v1.merge(
    traces_v2,
    on='merge_key',
    suffixes=('_v1', '_v2')
)

print(f"v1とv2の間で{len(merged)}の一致する例が見つかりました")

# 特定のメトリクスが改善しなかった例を見つける
regression_examples = []

for idx, row in merged.iterrows():
    v1_assessments = {a.name: a for a in row['assessments_v1']}
    v2_assessments = {a.name: a for a in row['assessments_v2']}

    # 各スコアラーについてリグレッションをチェック
    for scorer_name in ['follows_instructions', 'concise_communication', 'includes_next_steps', 'retrieval_groundedness']:
        v1_assessment = v1_assessments.get(scorer_name)
        v2_assessment = v2_assessments.get(scorer_name)

        if v1_assessment and v2_assessment:
            v1_val = v1_assessment.feedback.value
            v2_val = v2_assessment.feedback.value

            # メトリクスが悪化したかどうかをチェック（yes -> no）
            if v1_val == 'yes' and v2_val == 'no':
                regression_examples.append({
                    'index': idx,
                    'customer': row['request_v1']['customer_name'],
                    'instructions': row['request_v1']['user_instructions'],
                    'metric': scorer_name,
                    'v1_score': v1_val,
                    'v2_score': v2_val,
                    'v1_rationale': v1_assessment.rationale,
                    'v2_rationale': v2_assessment.rationale,
                    'v1_response': row['response_v1']['email'],
                    'v2_response': row['response_v2']['email']
                })

# リグレッションの例を表示
if regression_examples:
    print(f"\n=== {len(regression_examples)}のメトリクスリグレッションが見つかりました ===\n")

    # メトリクスごとにグループ化
    by_metric = {}
    for ex in regression_examples:
        metric = ex['metric']
        if metric not in by_metric:
            by_metric[metric] = []
        by_metric[metric].append(ex)

    # リグレッションが発生したメトリクスごとの例を表示
    for metric, examples in by_metric.items():
        print(f"\n{'='*80}")
        print(f"メトリクスリグレッション: {metric}")
        print(f"{'='*80}")

        # このメトリクスの最初の例を表示
        ex = examples[0]
        print(f"\n顧客: {ex['customer']}")
        print(f"指示: {ex['instructions']}")
        print(f"\nV1スコア: ✓ (合格)")
        print(f"V1理由: {ex['v1_rationale']}")
        print(f"\nV2スコア: ✗ (不合格)")
        print(f"V2理由: {ex['v2_rationale']}")

        if len(examples) > 1:
            print(f"\n(+{len(examples)-1}の例が{metric}リグレッションを含む)")
else:
    print("\n✓ メトリクスリグレッションは見つかりませんでした - V2はすべてのメトリクスを改善または維持しました！")

v1とv2の間で7の一致する例が見つかりました

=== 3のメトリクスリグレッションが見つかりました ===


メトリクスリグレッション: follows_instructions

顧客: Global Retail
指示: 四半期レビューの要約を送信

V1スコア: ✓ (合格)
V1理由: The user instruction was to send a summary of the quarterly review. The response provided is a follow-up email template that includes a brief summary of the meeting points discussed during the quarterly review. Therefore, the response adheres to the user instructions provided in the request.

V2スコア: ✗ (不合格)
V2理由: The user instruction was to send a summary of the quarterly review. The response includes an email that mentions a follow-up from a meeting and highlights satisfaction with the platform's performance during the quarterly review. However, it does not provide a clear summary of the quarterly review itself. Instead, it mentions a follow-up on pricing and does not focus on summarizing the review content. Therefore, the response does not fully adhere to the user instructions.

メトリクスリグレッション: concise_communication

顧客: Global Retail
指示